## Zaawansowane Metody Inteligencji Obliczeniowej
# Zadanie domowe 1
### Prowadzący: Michał Kempka, Marek Wydmuch
### Autor: Adam Marciniak + 141273

## Wprowadzenie

Całe zadanie jest oparte o różne wersje środowiska `VacuumEnvironemnt`, które rozważaliśmy na zajęciach.
Środowisko zaimplementowane jest w bibliotece aima3 (https://github.com/ArtificialIntelligenceToolkit/aima3),
która zawiera kod do książki "Artificial Intelligence: A Modern Approach".

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod) o ile nie napisano gdzieś inaczej.

In [1]:
# Zainstaluj bibliotekę OpenAI Gym
!pip install aima3 -q

     |████████████████████████████████| 150 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 60.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.18.3 requires networkx>=2.0, but you have networkx 1.11 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
# Zaimportuj wszystkie jego elementy
from aima3.agents import *

Wszystkie używane przez nas elementy biblioteki są zaimplementowane w pliku: https://github.com/ArtificialIntelligenceToolkit/aima3/blob/master/aima3/agents.py

# Zad. 1 - Cechy środowiska odkurzacza (1 pkt.)

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź:
Środowisko to jest:
- Częściowo obserwowalne - stan obecnego pokoju jest znany, ale nie wiemy, co jest w drugim.
- Jednoagentowe.
- Deterministyczne.
- Dyskretne:
  * mamy skończoną ilość czasu/kroków (środowisko kończy się po `x` krokach);
  * są tylko 4 możliwe akcje: {suck, left, right, noop};
  * jest tylko 8 możliwych kombinacji stanów (2 dla każdego pokoju: {dirty, clean} oraz 2 dla agenta: {left, right}).
- Model środowiska jest znany.

Tip: Możesz sprawdź implementacje środowiska w pliku podanym powyżej, lub wywnioskować cechy na wykonując poniższe fragmenty kodu.

In [3]:
# Stwórz nowe środowisko świata odkurzacza
env = TrivialVacuumEnvironment()

In [4]:
# Sprawdź aktualny status środowiska
env.status

{(0, 0): 'Clean', (1, 0): 'Clean'}

In [5]:
# Utwórz agenta refleksyjnego
agent = ReflexVacuumAgent()
agent.is_alive()

True

In [6]:
# Dodaj agenta do środowiska. Owijamy go w TraceAgent'a, żeby zobaczyć co robi.
env.add_thing(TraceAgent(agent))

In [7]:
# Zobacz gdzie jest agent
for loc in [loc_A, loc_B]:
    print('loc {0}: {1}'.format(loc, env.list_things_at(loc)))
# Lub:
agent.location

loc (0, 0): []
loc (1, 0): [<Agent>]


(1, 0)

In [8]:
# Wykonaj 10 kroków
env.run(10)

<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right


In [9]:
# Sprawdź jak środowisko oceniło jakość agenta.
agent.performance

-10

In [10]:
# Moglibyśmy ocenić oczekiwaną jakość agenta dokładniej..., ale tylko ją oszacujemy (1000 powtórzeń).
# Zakładamy, że symulacja trwa 50 kroków.

compare_agents(TrivialVacuumEnvironment, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent>, -38.824),
 (<function aima3.agents.ModelBasedVacuumAgent>, 9.16)]

# Zad. 2 - Cechy zmodyfikowanego środowisko odkurzacza (1 pkt).

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź:
Środowisko to jest:
- Częściowo obserwowalne - stan obecnego pokoju jest znany, ale nie wiemy, co jest w drugim.
- Jednoagentowe.
- Stochastyczne - z jednej strony za każdą akcję Agent zawsze dostanie tę samą karę lub nagrodę (determinizm), ale z drugiej strony losowa szansa na zabrudzenie sprawia, że ta sama akcja może doprowadzić do różnych stanów. Może na przykład dojść do sytuacji, w której Agent wyczyści pokój, ale w następnym kroku zaobserwuje, że pokój nadal jest brudny, mimo że powinien być czysty. Agent może też czekać przez parę kroków w czystym pokoju i raz zaobserwować pokój czysty, a innym razem brudny.
- Dyskretne:
  * nadal mamy skończoną ilość czasu/kroków (środowisko kończy się po `x` krokach);
  * nadal są tylko 4 możliwe akcje: {suck, left, right, noop}, chociaż tym razem mogą prowadzić do różnych stanów;
  * nadal jest tylko 8 możliwych kombinacji stanów (2 dla każdego pokoju: {dirty, clean} oraz 2 dla agenta: {left, right}).
- Model środowiska jest znany - znamy prawdopodobieństwo ponownego zabrudzenia, a tym samym rozkład prawdopodobieństw konsekwencji akcji Agenta.

In [11]:
# Rozszerzmy implementacje TrivialVacuumEnvironment

import random

class TrivialVacuumEnvironmentWithCats(TrivialVacuumEnvironment):
    def __init__(self, random_dirt_prob=0.05, seed=None):
        super(TrivialVacuumEnvironmentWithCats, self).__init__()
        self.random = random.Random(seed)
        self.random_dirt_prob = random_dirt_prob

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance; add dirt;
        Score 10 for each dirt cleaned; -1 for each move."""
        # Same as in case of TrivialVacuumEnvironment
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'

        # Cats can make either location dirty
        for loc in [loc_A, loc_B]:
            if self.random.random() < self.random_dirt_prob:
                self.status[loc] = 'Dirty'

In [12]:
# Przetestujmy domyślnych agentów w nowym środowisku

def env_factory():
    return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)

compare_agents(env_factory, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent>, 13.415),
 (<function aima3.agents.ModelBasedVacuumAgent>, 33.71)]

# Zad. 3 - Własny program agenta (8 pkt.)

Napisz program agenta, który będzie (średnio) dużo lepszy dla tego środowiska (50 kroków, z random_dirt_prob=0.05) niż ModelBaseVacuumAgent oraz ReflexVacuumAgent. Opisz działanie swojego programu, na podstawie jaki przesłanek on działa, jakbyś go zmodyfikował gdyby prawdopodobieństwo zabrudzenia pokoju (random_dirt_prob) się zmieniło?

Punktacja za wynik (sprawdzarka zrobi 50000 powtórzeń):
* \> 41: 1 pkt.
* \> 42: 2 pkt.
* \> 43: 3 pkt.
* \> 44: 4 pkt.
* \> 45: 5 pkt.
* \> 46: 6 pkt.

\+ 2 pkt. za opis.

#### Uwaga: nie zmieniaj nazwy klasy `MyVacuumAgent`. Nie dopisuj do komórki z klasą innego kodu. Możesz zdefiniować funkcje pomocnicze w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórkę z klasę o nazwie `MyVacuumAgent` do sprawdzenia).

In [13]:
# Klasa MyVacuumAgent wypełniona przykładowym kodem agenta z modelem

def MyVacuumAgent():
    random_dirt_prob = 0.05
    moves_left = 50
    time_since_jump = 50

    def calc_chance(p, T):
        # calculate the chance of cats making mess in the other room 
        chance = 0
        for i in range(T):
            chance += p * (1-p)**(i-1)
        return chance

    def program(percept):
        nonlocal random_dirt_prob
        nonlocal moves_left
        nonlocal time_since_jump
        moves_left -= 1
        time_since_jump += 1
        dirty_chance = calc_chance(random_dirt_prob, time_since_jump)

        location, status = percept

        if status == 'Dirty':
            return 'Suck'
        elif (moves_left == 0):
            return 'NoOP'
        elif (dirty_chance > 0.4) or (moves_left == 1 and dirty_chance > 0.2):
            time_since_jump = 0
            if location == loc_A:
                return 'Right'
            elif location == loc_B:
                return 'Left'
        else:
            return 'NoOP'
        
    return Agent(program)

W każdym kroku Agent oblicza sobie prawdopodobieństwo `dirty_chance`, z jakim może zastać bałagan po wykonaniu przejścia do innego pokoju. Do obliczeń wykorzystuje wartość prawdopodobieństwa `random_dirt_prob` oraz ilość kroków, która minęła od ostatnich odwiedzin tego pokoju - `time_since_jump`.

Początkowo Agent przechodził do drugiego pokoju po tym, jak wyliczona szansa przekraczała **50%**, jednak kiedy zdałem sobie sprawę, że czeka w takim przypadku około 13 kroków w jednym pokoju, postanowiłem obniżyć ten próg do **40%** (10 kroków). Oprócz tego, jeżeli agent wykonuje właśnie swoją przedostatnią rundę (pozostała mu jeszcze jedna), to może przejść do drugiego pokoju, jeżeli szansa na zastanie bałaganu wynosi więcej niż **20%** (5 kroków), żeby w swojej ostatniej turze spróbować posprzątać ewentualny bałagan. Agent nie może przejść do drugiego pokoju w przypadku, kiedy wykonuje właśnie swój ostatni ruch - środowisko zakończy się w następnym kroku, więc Agent i tak nie zdąży już posprzątać drugiego pokoju.

Nawet gdyby prawdopodobieństwo `random_dirt_prob` się zmieniło, wydaje mi się że Agent powinien nadal otrzymywać w miarę dobre wyniki bez wprowadzania dodatkwoych zmian. Gdyby kara za przechodzenie do drugiego pokoju wzrosła jakoś znacząco, pewnie podniósłbym odpowiednio progi procentowe. Gdyby zmniejszyła się ilość ruchów, zastanowiłbym się z kolei nad drobnym obniżeniem tych progów.

In [14]:
# Przetestuj swojego agenta
def env_factory():
    return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)

compare_agents(env_factory, [MyVacuumAgent], 100000, 50)

[(<function __main__.MyVacuumAgent>, 46.5543)]